In [1]:
pip install pandas numpy scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


In [42]:
import os

print(os.getcwd())                # zeigt deinen aktuellen Ordner
print(os.listdir("../data"))      # zeigt, was im data-Ordner liegt



c:\Users\mayte\Documents\GitHub\ML-Project-Spam\notebook
['archive.zip', 'my_data.csv', 'spam_data.csv', 's_data.csv']


In [43]:
# 📘 Schritt 1: Bibliotheken importieren
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


In [44]:
import os
os.getcwd()



'c:\\Users\\mayte\\Documents\\GitHub\\ML-Project-Spam\\notebook'

In [45]:
os.listdir()


['.ipynb_checkpoints', 'anaconda_projects', 'spam.ipynb']

In [46]:
import os
print(os.listdir("../data"))


['archive.zip', 'my_data.csv', 'spam_data.csv', 's_data.csv']


In [48]:
# 📗 Schritt 2: Datensatz laden
df = pd.read_csv("../data/my_data.csv")
df.head()



,label,message
0,spam,Congratulations! You won an iPhone 15. Click h...
1,spam,Urgent! Your bank account needs verification o...
2,spam,Free Bitcoins are waiting for you! Sign up now.
3,spam,Earn $5000 per week from home!
4,spam,You have a package delivery issue. Confirm you...


In [49]:
# 📙 Schritt 3: Daten vorbereiten
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

X = df['message']
y = df['label_num']

In [50]:
# 📒 Schritt 4: Trainings- / Testdaten aufteilen
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
# 📕 Schritt 5: Text in Zahlen umwandeln (TF-IDF) – VERBESSERT
vectorizer = TfidfVectorizer(
    ngram_range=(1, 2),  # 1- und 2-Wort-Kombinationen
    min_df=1             # auch seltene Wörter behalten
)

In [52]:
# TF-IDF anwenden
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [53]:
# 📔 Schritt 6: Modell trainieren
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

MultinomialNB()

In [54]:
def predict_message(msg):
    msg_lower = msg.lower()
    
    # manuelle Spam-Wörter, die im alten Datensatz fehlen
    spam_keywords = ["iphone", "congratulations", "free", "won", "winner", "click", "claim", "prize"]
    
    # einfache Regel: wenn eines dieser Wörter vorkommt → Spam
    if any(word in msg_lower for word in spam_keywords):
        return "🚫 SPAM (keyword detected)"
    
    # sonst Modell benutzen
    msg_tfidf = vectorizer.transform([msg])
    pred = model.predict(msg_tfidf)[0]
    
    return "🚫 SPAM" if pred == 1 else "✅ Kein Spam"


In [55]:
# 📘 Schritt 7: Modell testen
y_pred = model.predict(X_test_tfidf)
print("Genauigkeit:", accuracy_score(y_test, y_pred))
print("\nBericht:\n", classification_report(y_test, y_pred))

Genauigkeit: 0.95

Bericht:
               precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.92      1.00      0.96        12

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



In [ ]:
# 📘 Schritt 7: Modell testen
y_pred = model.predict(X_test_tfidf)
print("Genauigkeit:", accuracy_score(y_test, y_pred))
print("\nBericht:\n", classification_report(y_test, y_pred))

Genauigkeit: 0.95

Bericht:
               precision    recall  f1-score   support

           0       1.00      0.88      0.93         8
           1       0.92      1.00      0.96        12

    accuracy                           0.95        20
   macro avg       0.96      0.94      0.95        20
weighted avg       0.95      0.95      0.95        20



In [56]:
# 📗 Schritt 8: Eigenen Text testen
def predict_message(msg):
    msg_tfidf = vectorizer.transform([msg])
    pred = model.predict(msg_tfidf)[0]
    return "🚫 SPAM" if pred == 1 else "✅ Kein Spam"

In [57]:
print(predict_message("Congratulations! You've won a free iPhone!"))

🚫 SPAM


In [58]:
print(predict_message("Hey let's meet tomorrow"))

✅ Kein Spam


In [59]:
print(predict_message("You have a virus. Klick this link"))

🚫 SPAM
